In [23]:
import numpy as np
import pandas as pd
import nltk
from nltk.corpus import stopwords
import matplotlib.pyplot as plt
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer
import pickle
import time
import re
import pandas as pd

In [24]:
train_set = pd.read_csv("train.csv")
test_set = pd.read_csv("test.csv")  
df = pd.read_csv("dataset.csv")
# df["type_of_biasness"].value_counts()
df["bias_rating"].value_counts()

bias_rating
Left      3000
Right     3000
Center    1000
Name: count, dtype: int64

In [25]:
df["bias_rating"] = df["type_of_biasness"].map({
    "Center" : "Center",
    "Conservative" : "Right",
    "Capitalist" : "Right",
    "Nationalist" : "Right",
    "Liberal" : "Left",
    "Secular" : "Left",
    "Socialist" : "Left"
})

df["type_of_biasness"].dropna(inplace=True)
df["bias_rating"].dropna(inplace=True)
df.to_csv("dataset.csv", index=False)

In [26]:
train_set["bias_rating"].value_counts()

bias_rating
Right     800
Left      800
Center    800
Name: count, dtype: int64

In [27]:
from sklearn.model_selection import train_test_split
train_set, test_set = train_test_split(df, test_size=0.2, random_state=42, stratify=df["bias_rating"])


In [28]:
left = train_set[train_set["bias_rating"] == "Left"]
right = train_set[train_set["bias_rating"] == "Right"]
center = train_set[train_set["bias_rating"] == "Center"]

In [29]:
con = train_set[train_set['type_of_biasness']== "Conservative"]
lib = train_set[train_set['type_of_biasness']== "Liberal"]
cen = train_set[train_set['type_of_biasness']== "Center"]
soc = train_set[train_set['type_of_biasness']== "Socialist"]
nat = train_set[train_set['type_of_biasness']== "Nationalist"]
sec = train_set[train_set['type_of_biasness']== "Secular"]
cap = train_set[train_set['type_of_biasness']== "Capitalist"]

print(f"{len(con)} {len(lib)} {len(cen)} {len(soc)} {len(nat)} {len(sec)} {len(cap)}")

823 813 800 789 786 798 791


Conservative = 0
Nationalist = 1
Capitalist = 2
Center = 3
Liberal = 4
Secular = 5
Socialist = 6


In [30]:
left["label"] = 0
center["label"] = 1
right["label"] = 2

C:\Users\Azlan\AppData\Local\Temp\ipykernel_10092\1763482999.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  left["label"] = 0
C:\Users\Azlan\AppData\Local\Temp\ipykernel_10092\1763482999.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  center["label"] = 1
C:\Users\Azlan\AppData\Local\Temp\ipykernel_10092\1763482999.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documenta

In [15]:
con["label"] = 0
nat["label"] = 1
cap["label"] = 2
cen["label"] = 3
lib["label"] = 4
sec["label"] = 5
soc["label"] = 6

C:\Users\Azlan\AppData\Local\Temp\ipykernel_12852\3385725523.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  con["label"] = 0
C:\Users\Azlan\AppData\Local\Temp\ipykernel_12852\3385725523.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nat["label"] = 1
C:\Users\Azlan\AppData\Local\Temp\ipykernel_12852\3385725523.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation

In [31]:
data = pd.concat([left, center, right], axis=0)

data.reset_index(inplace=True)

In [32]:
data.head()

,index,text,type_of_biasness,bias_rating,label
0,6373,"washington — rep. alexandria ocasio-cortez, d-...",Liberal,Left,0
1,5678,house majority leader kevin mccarthy told repo...,Liberal,Left,0
2,4003,mitch mcconnell pointedly declared “case close...,Liberal,Left,0
3,4860,took security luxury just signed deal new four...,Socialist,Left,0
4,6226,republicans senate house democrats friday lead...,Socialist,Left,0


In this section we will prepare the dataset in order to use in SVM model. Before starting I want to explain processes with 1 line explanations

Cleaning Links: We'll clean all website links using regular expressions
Cleaning Everything Except Alphabetical and Numerical Characters: We'll clean unrelevant digits using regular expressions
Tokenizing and Lemmatizing: We'll split texts into their words and convert words to their base form (dogs=>dog)
Removing Stopwords: We'll remove words that have no special meaning (such as the,will,was)
Joining Prepared Strings: We'll join the words
Bag of Words or tfidf Approach: We'll create a bag of word or tf-idf. 

In [33]:
import nltk

In [34]:
clean = []
lemma = WordNetLemmatizer()
swords = stopwords.words("english")

for line in data["text"]:
    line = re.sub(r'http\S+', '', line)
    line = re.sub("[^a-zA-Z0-9 ]"," ",line)
    line = line.lower()
    line = nltk.word_tokenize(line)
    line = [lemma.lemmatize(word) for word in line if word not in swords]
    line = " ".join(line)
    clean.append(line)

In [35]:
for i in range(10):
    print(data["text"][i])
    print(clean[i])
    print()

washington — rep. alexandria ocasio-cortez, d-n.y., does not want president donald trump's former campaign chairman paul manafort to be held in solitary confinement, she said in a wednesday morning tweet.
ocasio-cortez was responding to a to a new york times report that manafort may be held in solitary confinement at the rikers island jail in new york.
noting that the notorious prison was located in her district, the freshman representative called solitary confinement a "license for gov torture and human rights violations."
"manafort should be released, along with...
washington rep alexandria ocasio cortez n want president donald trump former campaign chairman paul manafort held solitary confinement said wednesday morning tweet ocasio cortez responding new york time report manafort may held solitary confinement rikers island jail new york noting notorious prison located district freshman representative called solitary confinement license gov torture human right violation manafort relea

TFIDF vectorization

In [36]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(max_features=5000)
X = vectorizer.fit_transform(clean).toarray()
y = data["label"]

In [37]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [38]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(4480, 5000)
(1120, 5000)
(4480,)
(1120,)


Support Vector Machine Classifier Modeling
Everything is ready, now ve can fit our classifier.

In [39]:
from sklearn.svm import SVC

model = SVC()
model.fit(X_train, y_train)


,C,1.0
,kernel,'rbf'
,degree,3
,gamma,'scale'
,coef0,0.0
,shrinking,True
,probability,False
,tol,0.001
,cache_size,200
,class_weight,None
,verbose,False


In [40]:
predictions = model.predict(X_test)
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
print(confusion_matrix(y_test, predictions))
print(classification_report(y_test, predictions))
print(accuracy_score(y_test, predictions))

[[400   2  99]
 [ 77   8  71]
 [125   1 337]]
              precision    recall  f1-score   support

           0       0.66      0.80      0.73       501
           1       0.73      0.05      0.10       156
           2       0.66      0.73      0.69       463

    accuracy                           0.67      1120
   macro avg       0.69      0.53      0.51      1120
weighted avg       0.67      0.67      0.63      1120

0.6651785714285714


In [41]:
import joblib

joblib.dump(model, "svm_model_bias.pkl")
joblib.dump(vectorizer, "svm_bias_vectorizer.pkl")


['svm_bias_vectorizer.pkl']

KeyboardInterrupt: 

In [7]:
from scipy.sparse import hstack

data = pd.read_csv("dataset.csv")


In [9]:
data = pd.read_csv('dataset.csv')

data["bias_label"] = data["bias_rating"].map({
    "Left": 0,
    "Center": 1,
    "Right": 2
})

data["subtype_label"] = data["type_of_biasness"].map({
    "Conservative": 0,
    "Capitalist": 1,
    "Nationalist": 2,
    "Center" : 3,
    "Liberal" : 4,
    "Secular" : 5,
    "Socialist" : 6
})


In [10]:

import numpy as np

data["text"].dropna()
X_text = data["text"]
X_label = data["bias_label"]
y = data["subtype_label"]

In [13]:
clean = []
swords = stopwords.words("english")
lemma = WordNetLemmatizer()
for line in X_text:
    line = re.sub(r'http\S+', '', line)
    line = re.sub("[^a-zA-Z0-9 ]"," ",line)
    line = line.lower()
    line = nltk.word_tokenize(line)
    line = [lemma.lemmatize(word) for word in line if word not in swords]
    line = " ".join(line)
    clean.append(line)
X_text = clean

In [14]:
print(len(X_text))
print(len(X_label))
print(len(y))

7000
7000
7000


In [16]:
from sklearn.model_selection import train_test_split

In [17]:
X_text_train, X_text_test, X_label_train, X_label_test, y_train, y_test = train_test_split(
    X_text, X_label, y, test_size=0.2, random_state=42
)

vectorize = TfidfVectorizer(max_features=7000)
X_text_train_tfidf = vectorize.fit_transform(X_text_train)
X_text_test_tfidf = vectorize.transform(X_text_test)

X_train_combined = hstack((X_text_train_tfidf, X_label_train.values.reshape(-1,1)))
X_test_combined = hstack((X_text_test_tfidf, X_label_test.values.reshape(-1,1)))



In [18]:
from sklearn.svm import SVC

model = SVC()
model.fit(X_train_combined, y_train)

,C,1.0
,kernel,'rbf'
,degree,3
,gamma,'scale'
,coef0,0.0
,shrinking,True
,probability,False
,tol,0.001
,cache_size,200
,class_weight,None
,verbose,False


In [19]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

predictions = model.predict(X_test_combined)
print(confusion_matrix(y_test, predictions))
print(classification_report(y_test, predictions))
print(accuracy_score(y_test, predictions))

[[181   6  17   0   0   0   0]
 [  4 198   1   0   0   0   0]
 [ 21   1 149   0   0   0   0]
 [  0   0   0 181   0   0   0]
 [  0   0   0   0 153   3  59]
 [  0   0   0   0   1 198   9]
 [  0   0   0   0  19   7 192]]
              precision    recall  f1-score   support

           0       0.88      0.89      0.88       204
           1       0.97      0.98      0.97       203
           2       0.89      0.87      0.88       171
           3       1.00      1.00      1.00       181
           4       0.88      0.71      0.79       215
           5       0.95      0.95      0.95       208
           6       0.74      0.88      0.80       218

    accuracy                           0.89      1400
   macro avg       0.90      0.90      0.90      1400
weighted avg       0.90      0.89      0.89      1400

0.8942857142857142


In [21]:
import joblib

In [22]:
joblib.dump(model, "svm_model_subtype.pkl")
joblib.dump(vectorize, "svm_vectorizer_subtype.pkl")

['svm_vectorizer_subtype.pkl']